# Time series demo 1: AR(p) models on synthetic data, with known parameters


**Guest lecture**

Columbia IEOR 4729 : _Model Based Trading: Theory and Practice_

Q McCallum (http://qethanm.cc)

In [ ]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## %matplotlib inline

In [ ]:
np.random.seed( 4729 )

Use some `statsmodels` builtins to generate synthetic data that fits an autoregressive AR(3) model with the parameters: `0.5` , `0.25` , `-0.1`

Of note:

- Since we're using an ARMA generator, we only pass in parameters for the autoregressive model (and leave the params for the moving average model blank)
- Because of how `ArmaProcess` works, we pass in _negative_ values of our AR parameters.  This is only for AR parameters, not MA parameters.

In [ ]:
process_ar3 = sm.tsa.ArmaProcess(
    ar = [ 1 , - 0.5 , - 0.25 , 0.1 ] ,
    ma = [ 1 ] ,
)

In [ ]:
## Let the ARMA process warm up for a bit (`burnin` parameter)
## and generate 500 points befitting an AR(3) model of our stated parameters
y_ar3 = process_ar3.generate_sample(
    500 ,
    burnin = 1000
)

In [ ]:
## being lazy and building a temporary Series for plotting
_ = pd.Series( y_ar3 ).plot(
    title = "AR(3) data" ,
    figsize = ( 20 , 6 )
)

We already know that this is an AR(3) model (with a little noise)
which gives us the chance to test our tools:

In [ ]:
## acf -- autocorrelation function plot, aka "correlogram"
## Correlation ofthe series, against itself, at different lags.
## The light-blue area represents a confidence interval of 90% or 95%
## (depending on settings) ; any points outside of that range
## _might_ be significant.


_ = sm.graphics.tsa.plot_acf( y_ar3 )

In [ ]:
## pacf -- partial autocorrelation function plot
## Similar to autocorrelation function, but removes correlations based on previous lags.
## Once again, points outside of the shaded blue area _might_ be significant.

## For a deeper explanation of ACF vs PACF, see:
## https://towardsdatascience.com/significance-of-acf-and-pacf-plots-in-time-series-analysis-2fa11a5d10a8

_ = sm.graphics.tsa.plot_pacf( y_ar3 )

In [ ]:
## That's the data, and that hints at an AR(3) model.  Maybe.  Let's try to fit a model:

In [ ]:
## the `(3,0)` is a tuple that reflects the order of the AR and MA portions,
## respectively.  Since this is an AR(3) model that means MA(0), so, we pass `(3,0)`

model_ar3 = sm.tsa.ARMA( y_ar3 , (3,0) )
fit_ar3 = model_ar3.fit( trend="nc" , disp=0 )

In [ ]:
fit_ar3.summary()

In [ ]:
## for a fit:
## -  .fittedvalues --> values as generated by this set of params
## -  .resid --> residuals, aka "original data minus fitted values"

## and:
## - y_ar3 --> the original, synthetic data from the series

In [ ]:
_ = pd.DataFrame(
    {
        "y_ar"   : y_ar3 , 
        "model"  : fit_ar3.fittedvalues
    }
).plot(
    title = "AR(3) series: reality (y_ar) vs prediction (model)" ,
    figsize = ( 20 , 6 )
)

In [ ]:
## So far, so good ... but ... remember the rule: check your residuals.
_ = sm.graphics.tsa.plot_acf( fit_ar3.resid )
_ = sm.graphics.tsa.plot_pacf( fit_ar3.resid )

This shape on an ACF is a strong hint of white noise: notice, we only get a meaningful correlation at lag 0, which will always be 1.

Since it looks like a lot of random noise /white noise from here ... we're good!

The fit's `summary()` method included some diagnostic values.  Those are useful when _comparing to other fits._  On their own, they don't say a whole lot.

The big ones are:

- **AIC:** Akaike Information Criterion -- judges information loss in a model, based on the likelihood function; penalizes for number of parameters (to avoid overfitting)
- **BIC:** Bayesian Information Criterion -- similar to AIC, but penalizes differently for number of terms


We can test this.  Let's say we _didn't_ know, _a priori,_ that this was an AR(3) model. 

One way to check would be to try a handful of other fits.  We can use a `for` loop for this, and compare the AIC and BIC values that come back.

In [ ]:
## We already know this data reflects AR(3) but we'll show a simple way to
## test when we believe a series to be of _some_ kind of AR form:

ar_p_to_try = [
    (5,0) ,
    (4,0) ,
    (3,0) ,
    (2,0) ,
    (1,0) ,
]

param_search_results = []

print( "(Remember: lowest AIC wins)" )

for ar_p in ar_p_to_try :
    print( "trying parameters: {}".format( ar_p ) )
    model_testing = sm.tsa.ARMA( y_ar3 , ar_p ).fit( trend="nc" , disp=0 )
    ## model_testing = sm.tsa.AR( y_ar ).fit( maxlag=10 )

    print( "model params: {}".format( model_testing.params ) )
    print( "AIC:     {}".format( model_testing.aic ) )
    print( "BIC:     {}".format( model_testing.bic ) )
    print()

We see that the AR(3) model -- parameters `(3,0)` -- has the lowest AIC and BIC, which means that it's the best fit.

That doesn't mean that the lowest fit is the _absolute_ best possible model; it just means that it's the best for the values we tried.  It's entirely possible that a different model altogether would be a better fit.